# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

Using TensorFlow backend.


In [0]:
(trainX, trainY),(testX, testY) = tf.keras.datasets.cifar10.load_data()

In [0]:
trainX = trainX.astype('float32')
testX = testX.astype('float32')

In [4]:
trainX.shape

(50000, 32, 32, 3)

In [5]:
testX.shape

(10000, 32, 32, 3)

In [6]:
trainY.shape

(50000, 1)

In [7]:
testY.shape

(10000, 1)

In [0]:
trainY=np.reshape(trainY,(50000,))

In [0]:
testY=np.reshape(testY,(10000,))

In [0]:
trainX_lt5 = trainX[trainY<5]
trainX_ge5 = trainX[trainY>=5]

trainY_lt5 = trainY[trainY<5]
trainY_ge5 = trainY[trainY>=5]

testX_lt5 = testX[testY<5]
testX_ge5 = testX[testY>=5]

testY_lt5 = testY[testY<5]
testY_ge5 = testY[testY>=5]

In [11]:
print(trainX_lt5.shape)
print(trainX_ge5.shape)
print(trainY_lt5.shape)
print(trainY_ge5.shape)

(25000, 32, 32, 3)
(25000, 32, 32, 3)
(25000,)
(25000,)


In [12]:
print(testX_lt5.shape)
print(testX_ge5.shape)
print(testY_lt5.shape)
print(testY_ge5.shape)

(5000, 32, 32, 3)
(5000, 32, 32, 3)
(5000,)
(5000,)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
trainY_lt5 = tf.keras.utils.to_categorical(trainY_lt5, num_classes=5)
testY_lt5 = tf.keras.utils.to_categorical(testY_lt5, num_classes=5)

In [0]:
trainY_ge5 = trainY_ge5 - 5
testY_ge5 = testY_ge5 - 5

In [0]:
trainY_ge5 = tf.keras.utils.to_categorical(trainY_ge5, num_classes=5)
testY_ge5 = tf.keras.utils.to_categorical(testY_ge5, num_classes=5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
# Normalizing the data

trainX_lt5 = trainX_lt5/255
testX_lt5 = testX_lt5/255

trainX_ge5 = trainX_ge5/255
testX_ge5 = testX_ge5/255

In [0]:
tf.keras.backend.clear_session()
#Initialize model
model = tf.keras.models.Sequential()

In [18]:
#Add first convolutional layer
model.add(tf.keras.layers.Conv2D(32, #Number of filters 
                                 kernel_size=(3,3), #Size of the filter
                                 activation='relu',input_shape=(32,32,3)))
#Add second convolutional layer
model.add(tf.keras.layers.Conv2D(32, #Number of filters 
                                 kernel_size=(3,3), #Size of the filter
                                 activation='relu'))
#Add MaxPooling layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#Add third convolutional layer
model.add(tf.keras.layers.Conv2D(64, #Number of filters 
                                 kernel_size=(3,3), #Size of the filter
                                 activation='relu'))

#Add dropout layer
model.add(tf.keras.layers.Dropout(0.25))

#Add fourth convolutional layer
model.add(tf.keras.layers.Conv2D(64, #Number of filters 
                                 kernel_size=(3,3), #Size of the filter
                                 activation='relu'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
#Flatten the output
model.add(tf.keras.layers.Flatten())

#Dense layer
model.add(tf.keras.layers.Dense(128, activation='relu'))

#Add another dropout layer
model.add(tf.keras.layers.Dropout(0.25))

#Dense layer
model.add(tf.keras.layers.Dense(128, activation='relu'))

#Add another dropout layer
model.add(tf.keras.layers.Dropout(0.25))

#Dense layer
model.add(tf.keras.layers.Dense(128, activation='relu'))

#Output layer
model.add(tf.keras.layers.Dense(5, activation='softmax'))

In [0]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
__________

In [22]:
#Train the model
model.fit(trainX_lt5,trainY_lt5,          
          validation_data=(testX_lt5,testY_lt5),
          epochs=10,
          batch_size=32)

Train on 25000 samples, validate on 5000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
25000/25000 [==============================] - 8s 300us/sample - loss: 1.1427 - acc: 0.5147 - val_loss: 0.9312 - val_acc: 0.6118
Epoch 2/10
25000/25000 [==============================] - 6s 222us/sample - loss: 0.8894 - acc: 0.6461 - val_loss: 0.7938 - val_acc: 0.6896
Epoch 3/10
25000/25000 [==============================] - 6s 220us/sample - loss: 0.7810 - acc: 0.6994 - val_loss: 0.6943 - val_acc: 0.7366
Epoch 4/10
25000/25000 [==============================] - 6s 221us/sample - loss: 0.6829 - acc: 0.7412 - val_loss: 0.6863 - val_acc: 0.7354
Epoch 5/10
25000/25000 [==============================] - 6s 252us/sample - loss: 0.6099 - acc: 0.7725 - val_loss: 0.6269 - val_acc: 0.7640
Epoch 6/10
25000/25000 [==============================] - 6s 247us/sample - loss: 0.5453 - acc: 0.7985 - val_loss: 0.6121 - val_acc: 0.7678
Epoch 7/10
25000/25000 [==============================] - 5s 21

In [23]:
# Final Train Loss & Accuracy
model.evaluate(trainX_lt5,trainY_lt5)

25000/25000 [==============================] - 2s 74us/sample - loss: 0.2340 - acc: 0.9196


[0.23396147723197938, 0.9196]

In [24]:
# Final Validation Loss & Accuracy
model.evaluate(testX_lt5,testY_lt5)

5000/5000 [==============================] - 0s 78us/sample - loss: 0.5894 - acc: 0.8002


[0.5894270770549774, 0.8002]

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [25]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in model.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False
    
#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

conv2d
False
conv2d_1
False
max_pooling2d
False
conv2d_2
False
dropout
False
conv2d_3
False
flatten
False
dense
True
dropout_1
False
dense_1
True
dropout_2
False
dense_2
True
dense_3
True


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [26]:
#Train the model
model.fit(trainX_ge5,trainY_ge5,          
          validation_data=(testX_ge5,testY_ge5),
          epochs=10,
          batch_size=32)

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 6s 221us/sample - loss: 0.8718 - acc: 0.6744 - val_loss: 0.5920 - val_acc: 0.7802
Epoch 2/10
25000/25000 [==============================] - 5s 219us/sample - loss: 0.5357 - acc: 0.8069 - val_loss: 0.4581 - val_acc: 0.8334
Epoch 3/10
25000/25000 [==============================] - 6s 221us/sample - loss: 0.4365 - acc: 0.8447 - val_loss: 0.4454 - val_acc: 0.8362
Epoch 4/10
25000/25000 [==============================] - 6s 221us/sample - loss: 0.3842 - acc: 0.8633 - val_loss: 0.4226 - val_acc: 0.8488
Epoch 5/10
25000/25000 [==============================] - 6s 221us/sample - loss: 0.3375 - acc: 0.8804 - val_loss: 0.3787 - val_acc: 0.8694
Epoch 6/10
25000/25000 [==============================] - 6s 221us/sample - loss: 0.2872 - acc: 0.8982 - val_loss: 0.3728 - val_acc: 0.8698
Epoch 7/10
25000/25000 [==============================] - 6s 222us/sample - loss: 0.2603 - acc: 0.9072 - val_lo

In [27]:
# Final Train Loss & Accuracy
model.evaluate(trainX_ge5,trainY_ge5)

25000/25000 [==============================] - 2s 75us/sample - loss: 0.1030 - acc: 0.9647


[0.10300754014015198, 0.96472]

In [28]:
# Final Validation Loss & Accuracy
model.evaluate(testX_ge5,testY_ge5)

5000/5000 [==============================] - 0s 78us/sample - loss: 0.3733 - acc: 0.8814


[0.3732889340877533, 0.8814]

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [33]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [34]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [35]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score